# House Prices Group 5 
# (Aleksandar, Arthur, Cyrill, Selina)

## Load packages

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.size'] = 10

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Mute warnings (related to LogReg 'max_iter' param)
import warnings
warnings.filterwarnings('ignore')

# Function for Printing and Showing Confusion Matrix

In [ ]:
def print_results_crossvalidation(func, X_test, y_test):
  
  std_best_score = func.cv_results_["std_test_score"][func.best_index_]
  print(f"Best parameters: {func.best_params_}")
  print(f"Mean CV score: {func.best_score_:}")
  print(f"Standard deviation of CV score: {std_best_score:}")
  print("Test Score:".format(func.score(X_test, y_test)))

def report(y_true, y_pred):
    
  class_report = metrics.classification_report(y_true, y_pred)
  print(class_report)
  conf_matrix = confusion_matrix(y_true, y_pred, normalize = "all")
  conf_matrix = pd.DataFrame(conf_matrix, ["Class 0", "Class 1", " Class 2", "Class 3", " Class 4"],  ["Class 0", "Class 1", " Class 2", "Class 3", " Class 4"])
  sns.heatmap(conf_matrix, annot = True).set(xlabel = "Assigned Class", ylabel = "True Class", title = "Confusion Matrix")
     


## Load data and initial EDA

In [ ]:
# Load data
df = pd.read_csv("GroupProjectDataSet.csv", sep=',')
print('Shape of data frame:', df.shape)
df.head(10)

In [ ]:
df.describe()

### Overview

The data set consists of 1460 observations with 81 variables (including the target variable "(prize) class" and the id variable). 79 variables are descriptive variables that should explain Class.

Quantitative: 1stFlrSF, 2ndFlrSF, 3SsnPorch, BedroomAbvGr, BsmtFinSF1, BsmtFinSF2, BsmtFullBath, BsmtHalfBath, BsmtUnfSF, EnclosedPorch, Fireplaces, FullBath, GarageArea, GarageCars, GarageYrBlt, GrLivArea, HalfBath, KitchenAbvGr, LotArea, LotFrontage, LowQualFinSF, MSSubClass, MasVnrArea, MiscVal, MoSold, OpenPorchSF, OverallCond, OverallQual, PoolArea, ScreenPorch, TotRmsAbvGrd, TotalBsmtSF, WoodDeckSF, YearBuilt, YearRemodAdd, YrSold

Qualitative: Alley, BldgType, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, BsmtQual, CentralAir, Condition1, Condition2, Electrical, ExterCond, ExterQual, Exterior1st, Exterior2nd, Fence, FireplaceQu, Foundation, Functional, GarageCond, GarageFinish, GarageQual, GarageType, Heating, HeatingQC, HouseStyle, KitchenQual, LandContour, LandSlope, LotConfig, LotShape, MSZoning, MasVnrType, MiscFeature, Neighborhood, PavedDrive, PoolQC, RoofMatl, RoofStyle, SaleCondition, SaleType, Street, Utilities

## Handling Missing Data

In [ ]:
# Plot missing values

missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
missing.plot.bar()

In [ ]:
cols = df.columns[df.isna().any()]
df_nan = df[cols].copy()
df_nan['Class'] = df['Class']
df_nan.isna().sum() / df_nan.shape[0]

# Plot missing values 2.0
plt.figure(figsize=(10, 6))
sns.heatmap(df_nan.isna().transpose(),
            cmap="Blues",
            cbar_kws={'label': 'Missing Values'});

In [ ]:
# Percentage of missing values for the variables

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Share'])
missing_data.head(20)

19 variables have missing values. Of the 19 variables four (PoolQC, MiscFeature, Alley, Fence) have more than 50% missing data and one (FireplaceQu) with nearly 50% missing data. But often NA does not mean that there is no data available. Instead (especially for thecategorical variables) it means that the house is lacking this specific object. NA in the PoolQC variable means that there is no pool; NA in the Alley variable means that there is "no alley access". All the descriptions of which NA stand for non-available data and which stand for a missing trait can be found in the [data description](https://www.openml.org/search?type=data&sort=runs&id=42165&status=active).

The following variables have NAs that can be filled:

- PoolQC: Na = No Pool
- MiscFeature: Na = None
- Alley: NA = No alley access
- Fence: NA = No Fence
- FireplaceQu: NA = No Fireplace
- GarageCond: NA = No Garage
- GarageType: NA = No Garage
- GarageFinish: NA = No Garage
- GarageQual: NA = No Garage
- BsmtFinType2: NA = No Basement
- BsmtExposure: NA = No Basement
- BsmtQual: NA = No Basement
- BsmtCond: NA = No Basement
- BsmtFinType1: NA = No Basement


In [ ]:
# Filling missing values for variables where appropriate

df["PoolQC"] = df["PoolQC"].fillna(value = "No")
df["MiscFeature"] = df["MiscFeature"].fillna(value = "No")
df["Alley"] = df["Alley"].fillna(value = "No")
df["Fence"] = df["Fence"].fillna(value = "No")
df["FireplaceQu"] = df["FireplaceQu"].fillna(value = "No")
df["GarageCond"] = df["GarageCond"].fillna(value = "No")
df["GarageType"] = df["GarageType"].fillna(value = "No")
df["GarageFinish"] = df["GarageFinish"].fillna(value = "No")
df["GarageQual"] = df["GarageQual"].fillna(value = "No")
df["BsmtFinType2"] = df["BsmtFinType2"].fillna(value = "No")
df["BsmtExposure"] = df["BsmtExposure"].fillna(value = "No")
df["BsmtQual"] = df["BsmtQual"].fillna(value = "No")
df["BsmtCond"] = df["BsmtCond"].fillna(value = "No")
df["BsmtFinType1"] = df["BsmtFinType1"].fillna(value = "No")


In [ ]:
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
missing.plot.bar()

In [ ]:
# Percentage of missing values for the variables

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Share'])
missing_data.head(5)

For all but five variables we coud fill the missing data because with them NA indicates the lack of the corresponding trait. For LotFrontage we miss 17% of the values and 5.5% for GarageYrBlt. 

- LotFrontage ---> High Correlation with other variable?
- GarageYrBlt can probably be ignored since it highly correlates with YearBuilt. 
- MasVnrType and MasVnrArea have a strong correaltion with "YearBuilt" and "OverallQual" ---> Delete them?
- Electrical one missing value ---> Delete this observation or just leave it?



In [ ]:
# further data cleaning
df = df.dropna(axis='columns', thresh=1459)
df = df.dropna(axis='rows', how = "any")

## Feature Engineering

### Dealing with Categorical Features (Encoding Categorical Variables) / Splitting Into X and y

In [ ]:
# Numerical variables that should be handled as categorical variables
df = df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"}})
df = df.replace({"MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})

In [ ]:
# we see that MoSold was succesfully changed
df.head(10)

In [ ]:
df.info()

In [ ]:
# Asign columns to feature matrix X and response vector y
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]

X.head(5)

In [ ]:
y.head(5)

In [ ]:
X.shape

In [ ]:
# factorise the binary variables (no need to create two dummy variables)
# ---> Problem of Multicollinearity 
#Without this the get_dummies would create two variables CentralAir_y and CentralAir_n
X["StreetFac"] = X.Street.factorize()[0]
X["CentralAirFac"] = X.CentralAir.factorize()[0]

In [ ]:
# Factorize categorical values, assign output to X
# create (multiple) dummy variables for a categorical variable
# panda way

X = pd.get_dummies(X.iloc[:,:]) # not using ID
X.head()

In [ ]:
X.columns.values

### Partitioning of the Data Set Into Train and Test Set

We are using a 70/30 (training/testing) splitting. (The parameter `random_state=0` fixes the random split in a way such that results are reproducible.)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    stratify=y)

In [ ]:
X_train.info()

## Handling Outliers

In [ ]:
out = IsolationForest(random_state = 42).fit(X_train)
out_train = out.predict(X_train)
out_test = out.predict(X_test)


X_train_wout_out = X_train[np.where(out_train == 1, True, False)]
y_train_wout_out = y_train[np.where(out_train == 1, True, False)]
X_test_wout_out = X_test[np.where(out_test == 1, True, False)]
y_test_wout_out = y_test[np.where(out_test == 1, True, False)]

print("Training Set")
print("Shape including outliers: ", X_train.shape)
print("Shape excluding outliers: ", X_train_wout_out.shape)
print("Nr. of outliers removed: ", X_train.shape[0]-X_train_wout_out.shape[0])

print(50*"-")

print("Test Set")
print("Shape including outliers: ", X_test.shape)
print("Shape excluding outliers: ", X_test_wout_out.shape)
print("Nr. of outliers removed: ", X_test.shape[0]-X_test_wout_out.shape[0]) 

In [ ]:
# Using the data without the outliers for the models

# Traning Set
X_train = X_train_wout_out
y_train = y_train_wout_out

# Test Set
X_test = X_test_wout_out
y_test = y_test_wout_out


### Feature Scaling

In [ ]:
# Get cols to scale
cols_scl = X.columns.values[:]

# Apply MinMaxScaler on continuous columns only
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train[cols_scl])  # fit & transform
X_test_norm  = mms.transform(X_test[cols_scl])  # ONLY transform

In [ ]:
# Apply StandardScaler on continuous columns only
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train[cols_scl])  # fit & transform
X_test_std  = stdsc.transform(X_test[cols_scl])  # ONLY transform

## Assessing Target Variable "Class"

** Assess Class imbalance. You make your own assessment on potential effects of class-imbalance. **

In [ ]:
plt.figure(1); plt.title('Distribution of Class')
sns.histplot(data=y, discrete = True)

We see that our "Class" deviates from the normal distribution, is positively skewed and shows peakedness (cortosis).

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['Class'].skew())
print("Kurtosis: %f" % df['Class'].kurt())

# Decision Trees / Random Forest

## Decision Trees

In [ ]:
# Initializing decision tree
tree = DecisionTreeClassifier(max_depth=4, random_state = 42)
tree.fit(X_train, y_train)

# Performance metrics for training and test set
print('Train score: ', tree.score(X_train, y_train))
print('Test score: ', tree.score(X_test, y_test))

print(70*'-')

# Confusion matrix
y_pred = tree.predict(X_test)
print('Confusion matrix for Decision Tree: \n')
print("Left = Predicted; Top = Actual")
print(metrics.confusion_matrix(y_test, y_pred))

We see that 3 were predicted to be in class 4 but were actually in class 2. On was predicted to be in class 4 but was actually in class 1. 

### Visualizing Decision Tree
[Scikit-learn website for details.](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html).

In [ ]:
# Plot tree
plt.figure(figsize=(40, 23))
plot_tree(tree, filled=True, feature_names = list(X.columns), rounded=True, class_names=["0", "1", "2", "3", "4"],);

### Grid Search (Finding Hyperparameters)
Evaluates the model performance for each combination of hyperparameter to obtain the optimal combination of values from this set (Raschka (2015)).

In [ ]:
# get a list of all possible parameters
print(f"Parameters of the Decision Tree: {tree.get_params().keys()}")

In [ ]:
# k-Fold CV object (k = 5)
kFold = StratifiedKFold(n_splits=5)


####### Parameters of the Decision Tree under investigation #######

# In the initial tuning we included more values.
# But more values cause more computational effort.
# We only have a preselected list of values that include values around the best value.


##### Estimators #####

# Criterion
# The function to measure the quality of a split.
Criterion = np.array(["gini", "entropy"])

# Splitter 
# The strategy used to choose the split at each node.
Splitter = np.array(["best", "random"])

# Class weight
# Weights associated with classes in the form {class_label: weight}.
class_Weight = np.array([None, "balanced", "balanced_subsample"])

# Max depth
# The maximum depth of the tree. 
maxDepth = np.array([1, 5, 7, 8, 9, 10, 11, 12, 15, 18])

# Max features
# The number of features to consider when looking for the best split.
max_Features = np.array([None, "auto", "sqrt", "log2"])

# min_Samples_Split
# The minimum number of samples required to split an internal node.
min_Samples_Split = np.array([1, 2, 3, 4, 5])

# minSamplesLeaf
minSamplesLeaf = np.array([1, 2, 3, 4, 5])




In [ ]:
# hyperparameter to be tested
param_grid_tr = {"class_weight": class_Weight,
              #"criterion": Criterion,
              "max_depth": maxDepth,
              #"splitter": Splitter,
              #"max_features": max_Features,
              "min_samples_split": min_Samples_Split,
              "min_samples_leaf": minSamplesLeaf,
}

# grid search
tree_gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42, max_features = "sqrt"),
                  param_grid=param_grid_tr,
                  scoring="accuracy",
                  cv=kFold, n_jobs=-1)
tree_gs = tree_gs.fit(X_train, y_train)

print("Performance of Decision Tree")
print_results_crossvalidation(tree_gs, X_test, y_test)
y_pred = tree_gs.best_estimator_.predict(X_test)

report(y_test, y_pred)

In [ ]:
# Take best parameter
clf = tree_gs.best_estimator_

# Fitting the model with the best parameter
clf.fit(X_train, y_train)

# Print out score on Test dataset
print("Accuracy Test Set: {0: .4f}".format(clf.score(X_test, y_test)))

## Random Forest

### Grid Search (Finding Hyperparameters)
Evaluates the model performance for each combination of hyperparameter to obtain the optimal combination of values from this set (Raschka (2015)).

In [ ]:
# Initializing Classifier object
forest = RandomForestClassifier(n_estimators = 100, criterion="gini", random_state=42, n_jobs=-1)

# Get a list of all parameters of random forest
print(f"Parameters of Random Forest: {forest.get_params().keys()}")

In [ ]:
# k-Fold CV object (k = 5)
kFold = StratifiedKFold(n_splits=5)


####### Parameters for the Random Forest #######

# In the initial tuning we included more values.
# But more values cause more computational effort.
# We only have a preselected list of values that include values around the best value.


##### Estimators #####

# The number of trees in the forest.
n_Estimators = np.array([90, 95, 100, 105])

# Criterion
# The function to measure the quality of a split.
Criterion = np.array(["gini", "entropy"])

# Class weight
# Weights associated with classes in the form {class_label: weight}.
class_Weight = np.array([None, "balanced", "balanced_subsample"])

# Max depth
# The maximum depth of the tree. 
maxDepth = np.array([5, 10, 15, 20])

# Max features
# The number of features to consider when looking for the best split.
max_Features = np.array([None, "auto", "sqrt", "log2"])

# min_Samples_Split
# The minimum number of samples required to split an internal node.
min_Samples_Split = np.array([2, 3, 4])

# minSamplesLeaf
minSamplesLeaf = np.array([1, 2, 3])

# Bootstrap
BootStrap = np.array(["False", "True"])


In [ ]:
# hyperparameter to be tested (for computational reasons those where the default is the best are hashtaged)
param_grid_fo = {
              #"class_weight": class_Weight,
              #"criterion": Criterion,
              "n_estimators": n_Estimators,
              "max_depth": maxDepth,
              #"max_features": max_Features,
              "min_samples_split": min_Samples_Split,
              "min_samples_leaf": minSamplesLeaf,
              #"bootstrap": BootStrap
}

# grid search
forest_gs = GridSearchCV(estimator=RandomForestClassifier(random_state=0, n_jobs=-1, max_features = "sqrt"),
                  param_grid=param_grid_fo,
                  scoring="accuracy",
                  cv=kFold, n_jobs=-1)
forest_gs = forest_gs.fit(X_train, y_train)

print("Performance of Random Forest")
print_results_crossvalidation(forest_gs, X_test, y_test)
y_pred = forest_gs.best_estimator_.predict(X_test)

report(y_test, y_pred)


In [ ]:
# Take best parameter
clf = forest_gs.best_estimator_

# Fitting the model with the best parameter
clf.fit(X_train, y_train)

# Print out score on Test dataset
print("Accuracy Test Set: {0: .4f}".format(clf.score(X_test, y_test)))